In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from geopy import distance
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [2]:
tdataTRAIN=pd.read_csv(r'initial_condensed_data_TRAIN.csv')
tdataTEST=pd.read_csv(r'initial_condensed_data_TEST.csv')

In [3]:
tdata=pd.concat([tdataTRAIN,tdataTEST])
tdata=tdata[tdata.Lat_Long.isna()==False]


In [4]:

#basic feautures to consider when comparing houses:
#-size
#-Quality
# -Year Built
# -Basement size
# -garage or not
# -remodeled?


In [5]:
def seperatelat(latlong):
    try:
        return latlong[0]
    except:
        return ''
def seperatelong(latlong):
    try:
        return latlong[1]
    except:
        return ''
def cleanlatlong(latlong):
    try:
        return [float(i) for i in latlong.replace('[', '').replace(']', '').replace(',', '').split()]
    except:
        return ''


In [6]:
tdata.Lat_Long=tdata.Lat_Long.apply(lambda x: cleanlatlong(x))
tdata['Lat']=tdata.Lat_Long.apply(lambda x: seperatelat(x))
tdata['Long']=tdata.Lat_Long.apply(lambda x: seperatelong(x))
tdata.to_csv(r'Data_For_KNN_salescomps.csv', index=False)

In [7]:
#1st - find houses within x mi radius of house in q
#2nd- KNeighborsRegressor to identify closest points and possible to predict y 

In [39]:


def get_sales_comps(tdata,instance,Miles,numofneighbors,features=None,):

    #features to compare inst to other houses 
    if features== None:
        features=['GrLivArea',  'LotArea',  'OverallQual', 'YearBuilt',
        'TotalBsmtSF',
       'GarageArea',
       'Remodeled']
        
        
    ## find people within x mis
    def houses_within_x_mi(tdata,instance,Miles):
        instance_coords=[instance.Lat.iloc[0],instance.Long.iloc[0]]
        tdata['Dist_from_Inst']=tdata.apply(lambda x: distance.distance([x.Lat,x.Long], instance_coords).miles,axis=1)
        return tdata[tdata.Dist_from_Inst<=Miles]


    
    datasubset=houses_within_x_mi(tdata,instance,Miles)
    datasubsetSalePrices=datasubset.SalePrice
    instance=instance[features]
    datasubset=datasubset[features]
    untransformed_data=datasubset

    scaler=StandardScaler()
    scaler.fit(datasubset)
    datasubset=scaler.transform(datasubset)
    instance=scaler.transform(instance)

    neigh = NearestNeighbors(n_neighbors=numofneighbors)
    neigh.fit(datasubset)
    dist,ind=neigh.kneighbors(instance)
    
    Sales_Comps=untransformed_data.iloc[ind.tolist()[0]]
    Sales_Comps['SalePrice']=datasubsetSalePrices.iloc[ind.tolist()[0]]
    
    return Sales_Comps
    
    

In [40]:
pd.DataFrame({'Houses':list(range(10))})

,Houses
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [41]:
#testing it out
i=0
instance=tdata.iloc[[i]]
#nstance.Lat.iloc[0]
instance

,PID,GrLivArea,SalePrice,MSSubClass,LotArea,LotConfig,Neighborhood,Condition1,BldgType,OverallQual,...,WoodDeckBinary,HasPorch,FenceBinary,OverallCondBinary,Lat_Long,Dist_From_UoI,LotShape_IR,SlopedLot,Lat,Long
0,923204140,1632,143000,multi_fam,25000,Inside,Mitchel,Norm,2Fam,5,...,False,0,0,1,"[41.99308116084478, -93.60743936126562]",4.931155,0,1,41.993081,-93.607439


In [42]:
temptdata=tdata.drop([i])


features=['GrLivArea',   'LotArea',  'OverallQual', 'YearBuilt',
        'TotalBsmtSF',
       'GarageArea',
       'Remodeled']

get_sales_comps(temptdata,instance,2,numofneighbors=10,features=features)

<ipython-input-39-5c3c9c254399>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdata['Dist_from_Inst']=tdata.apply(lambda x: distance.distance([x.Lat,x.Long], instance_coords).miles,axis=1)
<ipython-input-39-5c3c9c254399>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sales_Comps['SalePrice']=datasubsetSalePrices.iloc[ind.tolist()[0]]


,GrLivArea,LotArea,OverallQual,YearBuilt,TotalBsmtSF,GarageArea,Remodeled,SalePrice
923,1625,23580,6,1979,1625.0,576.0,False,242500
1220,1188,26142,5,1962,1188.0,312.0,False,157900
736,1356,15957,6,1977,1244.0,528.0,False,188000
749,1728,12640,6,1976,1728.0,574.0,False,150900
394,1126,17400,5,1977,1126.0,484.0,False,160000
111,1040,25286,4,1963,1064.0,648.0,False,132250
869,1160,17979,5,1968,1113.0,257.0,False,152500
249,988,21750,5,1954,988.0,520.0,False,113000
698,1495,12890,6,1989,1495.0,438.0,False,150000
1561,1652,11836,5,1970,1652.0,928.0,False,146500
